# Imports

In [13]:
%load_ext autoreload
%autoreload 2

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from sklearn.datasets import *

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import copy

from data_structures.tree_classifier import TreeClassifier
import utils.utils
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prepare Data

In [ ]:
# Download all datasets from sklearn
for m in [fetch_olivetti_faces, fetch_20newsgroups_vectorized, fetch_lfw_people, fetch_lfw_pairs, fetch_covtype, fetch_rcv1, fetch_kddcup99, fetch_california_housing]:
    print(m)
    try:
        all_ = m()
        train = m(subset='train')
        test = m(subset='test')
    except:
        pass

In [14]:
# Download the data from two categories
cats = ['alt.atheism', 'sci.space']
ng_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=cats)
ng_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=cats)


vectorizer = TfidfVectorizer()
trans = vectorizer.fit(ng_train.data)
train_vectors = vectorizer.transform(ng_train.data)
test_vectors = vectorizer.transform(ng_test.data)
print("Number of datapoints: ", len(ng_train.data))
print("Number of features: ", train_vectors.shape[1])
print("Balance: ", np.sum(ng_train.target) / len(ng_train.target)) # 55-45, roughly balanced

N_COMPONENTS=100
pca = PCA(n_components=N_COMPONENTS)
pca.fit(train_vectors.toarray())
pca_train_vecs = pca.transform(train_vectors.toarray())
pca_test_vecs = pca.transform(test_vectors.toarray())

classes_arr = np.unique(ng_train.target)
classes = utils.utils.class_to_idx(classes_arr)

Number of datapoints:  1073
Number of features:  18217
Balance:  0.5526561043802423


# Compare our implementation's accuracy to sklearn

In [15]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(pca_train_vecs,ng_train.target)
print("sklearn Decision Tree Accuracy:", np.mean(dt.predict(pca_test_vecs) == ng_test.target))

#cross_val_score(dt, pca_train_vecs, ng_train.target, cv=10).mean()

sklearn Decision Tree Accuracy: 0.7812061711079944


In [16]:
rf = RandomForestClassifier(random_state=0)
rf.fit(pca_train_vecs,ng_train.target)
print("sklearn Random Forest Accuracy:", np.mean(rf.predict(pca_test_vecs) == ng_test.target))

#cross_val_score(rf, pca_train_vecs, ng_train.target, cv=10).mean()

sklearn Random Forest Accuracy: 0.8022440392706872


In [17]:
tc = TreeClassifier(data=pca_train_vecs, labels=ng_train.target, max_depth=5, classes=classes, verbose=False)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)

Train accuracy: 0.9086672879776329
Test accuracy: 0.7812061711079944
Num queries: 5271
Runtime: 21.913602113723755


In [18]:
tc = TreeClassifier(data=pca_train_vecs, labels=ng_train.target, max_depth=5, classes=classes, solver="EXACT", verbose=False)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)

Train accuracy: 0.9086672879776329
Test accuracy: 0.7812061711079944
Num queries: 5344
Runtime: 12.790964126586914


# Make the dataset huge

In [19]:
doublings = 4
pca_train_vecs_huge = copy.deepcopy(pca_train_vecs)
pca_train_labels_huge = copy.deepcopy(ng_train.target)
print(pca_train_vecs_huge.shape)
for i in range(doublings):
    pca_train_vecs_huge = np.concatenate((pca_train_vecs_huge, pca_train_vecs_huge))
    pca_train_labels_huge = np.concatenate((pca_train_labels_huge, pca_train_labels_huge))
print(pca_train_vecs_huge.shape)

(1073, 100)
(17168, 100)


In [20]:
tc = TreeClassifier(data=pca_train_vecs_huge, labels=pca_train_labels_huge, max_depth=2, classes=classes, verbose=True, random_state=0)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)
tc.tree_print()

Calculated split with 6600 queries
Calculated split with 6576 queries
Calculated split with 8500 queries
Fitting finished
Train accuracy: 0.7996272134203168
Test accuracy: 0.7531556802244039
Num queries: 21676
Runtime: 12.382985830307007
|--- feature_1 <= -0.02424286634709316
|   |--- feature_18 <= 0.028246263503167335
|   |   |--- class: 0
|   |--- feature_18 > 0.028246263503167335
|   |   |--- class: 0
|--- feature_1 > -0.02424286634709316
|   |--- feature_3 <= -0.016323886201712157
|   |   |--- class: 1
|   |--- feature_3 > -0.016323886201712157
|   |   |--- class: 1




In [21]:
# We should implement vanilla TreeClassifier --> which uses identity bins

tc = TreeClassifier(data=pca_train_vecs_huge, labels=pca_train_labels_huge, max_depth=2, classes=classes, solver="EXACT", verbose=True, random_state=0)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)
tc.tree_print()

Calculated split with 17168 queries
Calculated split with 6576 queries
Calculated split with 10592 queries
Fitting finished
Train accuracy: 0.7996272134203168
Test accuracy: 0.7531556802244039
Num queries: 34336
Runtime: 21.095016956329346
|--- feature_1 <= -0.02424286634709316
|   |--- feature_18 <= 0.028246263503167335
|   |   |--- class: 0
|   |--- feature_18 > 0.028246263503167335
|   |   |--- class: 0
|--- feature_1 > -0.02424286634709316
|   |--- feature_3 <= -0.016323886201712157
|   |   |--- class: 1
|   |--- feature_3 > -0.016323886201712157
|   |   |--- class: 1




# Verify our implementation of baseline models agrees with sklearn's in terms of accuracy

# Classification:

In [ ]:
from data_structures.wrappers.random_forest_classifier import RandomForestClassifier as RFC_ours
from data_structures.wrappers.extremely_random_forest_classifier import ExtremelyRandomForestClassifier as ERFC_ours

from data_structures.wrappers.random_forest_regressor import RandomForestRegressor as RFR_ours
from data_structures.wrappers.extremely_random_forest_regressor import ExtremelyRandomForestRegressor as ERFR_ours

from sklearn.ensemble import RandomForestClassifier as RFC_sklearn
from sklearn.ensemble import ExtraTreesClassifier as ERFC_sklearn

from sklearn.ensemble import RandomForestRegressor as RFR_sklearn
from sklearn.ensemble import ExtraTreesRegressor as ERFR_sklearn

from utils.constants import GINI, BEST, EXACT, MSE

# TODO(@motiwari): Allow for gradient boosted comparisons as well
def compare_accuracies(
    compare: str = "RFC",
    train_data: np.ndarray = None,
    train_targets: np.ndarray = None,
    test_data: np.ndarray = None,
    test_targets: np.ndarray = None,
    num_seeds: int = 10,
) -> bool:
    our_train_accs = []
    our_test_accs = []
    their_train_accs = []
    their_test_accs = []
    for seed in range(num_seeds):
        # Ok to have n_jobs = -1 throughout?
        if compare == "RFC":
            our_model = RFC_ours(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = RFC_sklearn(n_estimators=5, criterion='gini', max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, n_jobs=-1, random_state=seed, verbose=0)
        elif compare == "ERFC":
            our_model = ERFC_ours(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, num_bins=None, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = ERFC_sklearn(n_estimators=5, criterion='gini', max_depth=5, min_samples_split=2, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=False, n_jobs=-1, random_state=seed, verbose=0)
        elif compare == "RFR":
            our_model = RFR_ours(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=MSE, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = RFR_sklearn(n_estimators=5, criterion='squared_error', max_depth=5, min_samples_split=2, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, n_jobs=-1, random_state=seed, verbose=0)    
        elif compare == "ERFR":
            our_model = ERFR_ours(data=train_data, labels=train_targets, n_estimators=5, max_depth=5, num_bins=None, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=MSE, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = ERFR_sklearn(n_estimators=5, criterion='squared_error', max_depth=5, min_samples_split=2, max_features='auto', min_impurity_decrease=0.0, bootstrap=False, n_jobs=-1, random_state=seed, verbose=0)
        else:
            raise NotImplementedError("Need to decide what models to compare")
        
        our_model.fit()
        their_model.fit(train_data, train_targets)

        if compare == "RFC" or compare == "ERFC" or compare == "HRFC":
            our_train_acc = np.mean(our_model.predict_batch(train_data)[0] == train_targets)
            our_test_acc = np.mean(our_model.predict_batch(test_data)[0] == test_targets)
            their_train_acc = np.mean(their_model.predict(train_data) == train_targets)
            their_test_acc = np.mean(their_model.predict(test_data) == test_targets)
        elif compare == "RFR" or compare == "ERFR" or compare == "HRFR":
            our_train_acc = np.mean((our_model.predict_batch(train_data) - train_targets)**2)
            our_test_acc = np.mean((our_model.predict_batch(test_data) - test_targets)**2)
            their_train_acc = np.mean((our_model.predict_batch(train_data) - train_targets)**2)
            their_test_acc = np.mean((our_model.predict_batch(test_data) - test_targets)**2)

        print("(Ours) Train accuracy:", our_train_acc)
        print("(Ours) Test accuracy:", our_test_acc)
        print("(Theirs) Train accuracy:", their_train_acc)
        print("(Theirs) Test accuracy:", their_test_acc)
        print("-" * 30)

        our_train_accs.append(our_train_acc)
        our_test_accs.append(our_test_acc)
        their_train_accs.append(their_train_acc)
        their_test_accs.append(their_test_acc)
    
    our_avg_train = np.mean(our_train_accs)
    our_std_train = np.std(our_train_accs)

    our_avg_test = np.mean(our_test_accs)
    our_std_test = np.std(our_test_accs)
    
    their_avg_train = np.mean(their_train_accs)
    their_std_train = np.std(their_train_accs)
    
    their_avg_test = np.mean(their_test_accs)
    their_std_test = np.std(their_test_accs)
    
    # See if confidence intervals overlap
    overlap = np.abs(their_avg_test - our_avg_test) < their_std_test + our_std_test
    return overlap, our_avg_train, our_std_train, our_avg_test, our_std_test, their_avg_train, their_std_train, their_avg_test, their_std_test

In [ ]:
train_data = pca_train_vecs
train_labels = ng_train.target
test_data = pca_test_vecs
test_labels = ng_test.target

compare_accuracies("RFC", train_data, train_labels, test_data, test_labels)

In [ ]:
compare_accuracies("ERFC", train_data, train_labels, test_data, test_labels)

# Regression: 

### Weirdly, we get exactly the same results as sklearn for all seeds. We should investigate whether this is a bug

In [ ]:
np.random.seed(0)

data, targets = fetch_california_housing(return_X_y=True)
random_idcs = np.random.choice(len(data), size=len(data),replace=False)
data = data[random_idcs]
targets = targets[random_idcs]

TRAIN_TEST_SPLIT = 16000
train_data = data[:TRAIN_TEST_SPLIT]
train_targets = targets[:TRAIN_TEST_SPLIT]

test_data = data[TRAIN_TEST_SPLIT:]
test_targets = targets[TRAIN_TEST_SPLIT:]

In [ ]:
compare_accuracies("RFR", train_data, train_targets, test_data, test_targets)

In [ ]:
compare_accuracies("ERFR", train_data, train_targets, test_data, test_targets)

# Verify MAB solvers are faster in speed but have no change in accuracy

# Classification

In [22]:
import time
from typing import Any, Tuple

# Classification #
# Vanilla random forest + H + GB + GBH
from data_structures.wrappers.random_forest_classifier import RandomForestClassifier as RFC
from data_structures.wrappers.histogram_random_forest_classifier import HistogramRandomForestClassifier as HRFC
from data_structures.wrappers.gradient_boosted_random_forest_classifier import GradientBoostedRandomForestClassifier as GBRFC
from data_structures.wrappers.gradient_boosted_histogram_random_forest_classifier import GradientBoostedHistogramRandomForestClassifier as GBHRFC

# Extremely random forest + GB (already histogrammed)
from data_structures.wrappers.extremely_random_forest_classifier import ExtremelyRandomForestClassifier as ERFC
from data_structures.wrappers.gradient_boosted_extremely_random_forest_classifier import GradientBoostedExtremelyRandomForestClassifier as GBERFC

# Random patches + H + GB + GBH
from data_structures.wrappers.random_patches_classifier import RandomPatchesClassifier as RPC
from data_structures.wrappers.histogram_random_patches_classifier import HistogramRandomPatchesClassifier as HRPC
from data_structures.wrappers.gradient_boosted_random_patches_classifier import GradientBoostedRandomPatchesClassifier as GBRPC
from data_structures.wrappers.histogram_random_patches_classifier import HistogramRandomPatchesClassifier as HBRPC


# Regression #
# Vanilla random forest + H + GB + GBH
from data_structures.wrappers.random_forest_regressor import RandomForestRegressor as RFR
from data_structures.wrappers.histogram_random_forest_regressor import HistogramRandomForestRegressor as HRFR
from data_structures.wrappers.gradient_boosted_random_forest_regressor import GradientBoostedRandomForestRegressor as GBRFR
from data_structures.wrappers.gradient_boosted_histogram_random_forest_regressor import GradientBoostedHistogramRandomForestRegressor as GBHRFR

# Extremely random forest + GB (already histogrammed)
from data_structures.wrappers.extremely_random_forest_regressor import ExtremelyRandomForestRegressor as ERFR
from data_structures.wrappers.gradient_boosted_extremely_random_forest_regressor import GradientBoostedExtremelyRandomForestRegressor as GBERFR

# Random patches + H + GB + GBH
from data_structures.wrappers.random_patches_regressor import RandomPatchesRegressor as RPR
from data_structures.wrappers.histogram_random_patches_regressor import HistogramRandomPatchesRegressor as HRPR
from data_structures.wrappers.gradient_boosted_random_patches_regressor import GradientBoostedRandomPatchesRegressor as GBRPR
from data_structures.wrappers.histogram_random_patches_regressor import HistogramRandomPatchesRegressor as HBRPR

In [23]:
def time_measured_fit(
    model: Any,
) -> float:
    """
    Returns wall clock time of training the model, in seconds.
    
    Has a side effect: trains the model.
    """
    start = time.time()
    model.fit()
    end = time.time()
    return end - start

In [27]:
from utils.constants import GINI, BEST, EXACT, MAB, MSE


def compare_runtimes(
    compare: str = "HRFC",
    train_data: np.ndarray = None,
    train_targets: np.ndarray = None,
    test_data: np.ndarray = None,
    test_targets: np.ndarray = None,
    num_seeds: int = 10,
) -> bool:
    # Runtimes
    our_train_times = []
    their_train_times = []
    
    # For accuracies
    our_train_accs = []
    our_test_accs = []
    their_train_accs = []
    their_test_accs = []
    for seed in range(num_seeds):
        # Ok to have n_jobs = -1 throughout?
        if compare == "HRFC":
            our_model = HRFC(data=train_data, labels=train_targets, n_estimators=1, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=MAB, random_state=seed, verbose=False)
            their_model = HRFC(data=train_data, labels=train_targets, n_estimators=1, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
        else:
            raise NotImplementedError("Need to decide what models to compare")
        
        assert 'sklearn' not in their_model.__module__, "Cannot use sklearn models for runtime comparisons"
        
        our_runtime = time_measured_fit(our_model)
        our_train_times.append(our_runtime)

        their_runtime = time_measured_fit(their_model)
        their_train_times.append(their_runtime)

        if compare == "RFC" or compare == "ERFC" or compare == "HRFC":
            our_train_acc = np.mean(our_model.predict_batch(train_data)[0] == train_targets)
            our_test_acc = np.mean(our_model.predict_batch(test_data)[0] == test_targets)
            their_train_acc = np.mean(their_model.predict_batch(train_data)[0] == train_targets)
            their_test_acc = np.mean(their_model.predict_batch(test_data)[0] == test_targets)

        print("(Ours) Train accuracy:", our_train_acc)
        print("(Ours) Test accuracy:", our_test_acc)
        print("(Theirs) Train accuracy:", their_train_acc)
        print("(Theirs) Test accuracy:", their_test_acc)
        print("*" * 30)
        print("(Ours) Runtime:", our_runtime)
        print("(Theirs) Runtime:", their_runtime)
        print("-" * 30)

        our_train_accs.append(our_train_acc)
        our_test_accs.append(our_test_acc)
        their_train_accs.append(their_train_acc)
        their_test_accs.append(their_test_acc)
    
    # For accuracies
    our_avg_train = np.mean(our_train_accs)
    our_std_train = np.std(our_train_accs)

    our_avg_test = np.mean(our_test_accs)
    our_std_test = np.std(our_test_accs)
    
    their_avg_train = np.mean(their_train_accs)
    their_std_train = np.std(their_train_accs)
    
    their_avg_test = np.mean(their_test_accs)
    their_std_test = np.std(their_test_accs)
    
    # For runtimes
    our_avg_train_time = np.mean(our_train_times)
    our_std_train_time = np.std(our_train_times)
    
    their_avg_train_time = np.mean(their_train_times)
    their_std_train_time = np.std(their_train_times)
    
    
    # See if confidence intervals overlap
    overlap = np.abs(their_avg_test - our_avg_test) < their_std_test + our_std_test
    return overlap, our_avg_train, our_std_train, our_avg_test, our_std_test, their_avg_train, their_std_train, their_avg_test, their_std_test, our_avg_train_time, our_std_train_time, their_avg_train_time, their_std_train_time

In [28]:
test_labels = ng_test.target
compare_runtimes("HRFC", pca_train_vecs_huge, pca_train_labels_huge, pca_test_vecs, test_labels)

(Ours) Train accuracy: 0.7949673811742777
(Ours) Test accuracy: 0.7279102384291725
(Theirs) Train accuracy: 0.6514445479962722
(Theirs) Test accuracy: 0.5988779803646563
******************************
(Ours) Runtime: 4.293490886688232
(Theirs) Runtime: 4.669025897979736
------------------------------
(Ours) Train accuracy: 0.8313140726933831
(Ours) Test accuracy: 0.7643758765778401
(Theirs) Train accuracy: 0.7260018639328985
(Theirs) Test accuracy: 0.6297335203366059
******************************
(Ours) Runtime: 6.979787111282349
(Theirs) Runtime: 4.930572748184204
------------------------------
(Ours) Train accuracy: 0.6971109040074557
(Ours) Test accuracy: 0.6255259467040674
(Theirs) Train accuracy: 0.798695246971109
(Theirs) Test accuracy: 0.7503506311360448
******************************
(Ours) Runtime: 2.8953399658203125
(Theirs) Runtime: 4.9914870262146
------------------------------
(Ours) Train accuracy: 0.6915191053122087
(Ours) Test accuracy: 0.6016830294530154
(Theirs) Trai

(True,
 0.7306616961789375,
 0.0587449436343466,
 0.667461430575035,
 0.060469630618275004,
 0.7458527493010252,
 0.05584967868807881,
 0.6782608695652175,
 0.05223043583543516,
 4.139358592033386,
 1.2648541739642905,
 4.936821484565735,
 0.11075324854478229)